In [ ]:
import prehook

prehook.execute_prehook()

In [ ]:
import hook
import prehook
import database_handler
import lookups
import datetime

db_session = database_handler.create_connection()
# hook.create_etl_checkpoint(db_session)

In [ ]:
hook.return_lookup_items_as_dict(lookups.IncrementalField)

In [ ]:
import hook
import prehook
import database_handler
import lookups
import datetime

db_session = database_handler.create_connection()
etl_date = datetime.datetime(1900,1,1)
#SourceName is schema name
source_name = lookups.SourceName.DVD_RENTAL.value
#This will get you a list of tables inside this schema that are mentioned in SQLTablesToReplicate
tables = prehook.return_tables_by_schema(source_name)
#This will return 
incremental_date_dict = hook.return_lookup_items_as_dict(lookups.IncrementalField)
# for table in tables:
staging_query = f"""
        SELECT * FROM {source_name}.{tables[0]} WHERE {incremental_date_dict.get(tables[0])} >= '{etl_date}'
""" 
staging_df = database_handler.return_data_as_df(db_session= db_session, input_type= lookups.InputTypes.SQL, file_executor= staging_query)
dst_table = f"stg_{source_name}_{tables[0]}"
insert_stmt = database_handler.return_insert_into_sql_statement_from_df(staging_df, 'dw_reporting', dst_table)
for insert in insert_stmt:
        database_handler.execute_query(db_session=db_session, query= insert)

In [2]:
import misc_handler
df = misc_handler.return_match_df_from_web(93335,93336)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   match_id              1 non-null      int64 
 1   date                  1 non-null      object
 2   home_team             1 non-null      object
 3   away_team             1 non-null      object
 4   home_score            0 non-null      object
 5   away_score            0 non-null      object
 6   home_possession_%     0 non-null      object
 7   away_possession_%     0 non-null      object
 8   home_shots_on_target  0 non-null      object
 9   away_shots_on_target  0 non-null      object
 10  home_shots            0 non-null      object
 11  away_shots            0 non-null      object
 12  home_touches          0 non-null      object
 13  away_touches          0 non-null      object
 14  home_passes           0 non-null      object
 15  away_passes           0 non-null      object

In [1]:
from prehook import first_time_web_scraping
first_time_web_scraping()

WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: headless chrome=117.0.5938.62)
Stacktrace:
	GetHandleVerifier [0x00007FF6BF857892+54818]
	(No symbol) [0x00007FF6BF7C6AC2]
	(No symbol) [0x00007FF6BF67DA3B]
	(No symbol) [0x00007FF6BF679853]
	(No symbol) [0x00007FF6BF66CB9C]
	(No symbol) [0x00007FF6BF66DDF7]
	(No symbol) [0x00007FF6BF66CF95]
	(No symbol) [0x00007FF6BF66C00E]
	(No symbol) [0x00007FF6BF66BFB3]
	(No symbol) [0x00007FF6BF66A82E]
	(No symbol) [0x00007FF6BF66B123]
	(No symbol) [0x00007FF6BF67FE4F]
	(No symbol) [0x00007FF6BF6F7917]
	(No symbol) [0x00007FF6BF6DEAAA]
	(No symbol) [0x00007FF6BF6F75A2]
	(No symbol) [0x00007FF6BF6DE883]
	(No symbol) [0x00007FF6BF6B3691]
	(No symbol) [0x00007FF6BF6B48D4]
	GetHandleVerifier [0x00007FF6BFBBB992+3610402]
	GetHandleVerifier [0x00007FF6BFC11860+3962352]
	GetHandleVerifier [0x00007FF6BFC09D4F+3930847]
	GetHandleVerifier [0x00007FF6BF8F3646+693206]
	(No symbol) [0x00007FF6BF7D1628]
	(No symbol) [0x0000